In [1]:
import os
from copy import deepcopy

from tqdm import tqdm
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

from Environment import Environment, make_one_hot, give_mapping
from Networks import UserActor, AsstActor, CentralizedCritic

In [2]:
def give_prev_steps(prev_steps_assist, steps):
    prev_steps_assist = [[0,0,0,0,-1,-1] for i in range(steps-1)]
    return prev_steps_assist

In [3]:
class Agent:
    def __init__(self):
        self.user_actor = UserActor()
        self.asst_actor = AsstActor()
        self.critic = CentralizedCritic()
        self.optimizer_actors = tf.keras.optimizers.Adam(lr = 0.0001)
        self.optimizer_critic = tf.keras.optimizers.Adam(lr = 0.0002)
        self.huber_loss = tf.keras.losses.Huber()
        self.memory_len = 4
        
        self.gamma = 0.90
        self.env = Environment()
        self.env.cells = np.array([[0.7, 0.1], [0.1, 0.1], [0.5, 0.7], [0.6, 0.2], [0.7, 0.4], [0.2, 0.9]])
        self.env_cell_mapping = give_mapping(self.env.cells)
        self.env_cell_mapping = self.env_cell_mapping[np.newaxis, :, :, np.newaxis]
        self.eps = 10e-6
    
    def learn(self):
        env = self.env
        max_steps = 50
        running_reward = 0
        reached = 0
        best = 0
        
        for epoch in tqdm(range(100000)):
#             input_As = [] #Input of user agent
#             input_Bs = [] #Input of assitant agent/output of user agent
#             input_Cs = [] #Icon layout
#             input_Ds = [] #Output of assitant agent
            
            user_action_probs_history = []
            asst_action_probs_history = []
            critic_value_history = []
            rewards_history = []
            returns = [] #Returns
            
            done = False
            episode_reward = 0
            start, dest = env.give_start_dest()
            ob_user = [start[0], start[1], dest[0], dest[1]]
            prev_steps_assist = []
            prev_steps_assist = give_prev_steps(prev_steps_assist, self.memory_len)
            step = 0
            episode_reward = 0
            
            with tf.GradientTape(persistent = True) as tape:
                while not done and step<max_steps:
                    curr_loc = ob_user[:2]
                    target_loc = ob_user[2:4]
                    step+=1
                    ob_user = np.array(ob_user)[np.newaxis]
                    user_probs = self.user_actor.model(ob_user)
                    user_action = np.random.choice(4, p=np.squeeze(user_probs))
                    user_action_probs_history.append(tf.math.log(user_probs[0, user_action]))

                    action_user_one_hot = make_one_hot(user_action, 4)

                    ob_assist = [action_user_one_hot + curr_loc] 
                    ob_assist = prev_steps_assist + ob_assist
                    ob_assist = np.array(ob_assist)[np.newaxis]
                    
                    asst_probs = self.asst_actor.model([ob_assist, self.env_cell_mapping])
                    asst_action = np.random.choice(4, p=np.squeeze(asst_probs))
                    asst_action_probs_history.append(tf.math.log(asst_probs[0, asst_action]))
                    
                    asst_output_one_hot = np.array(make_one_hot(asst_action, 4))[np.newaxis]
                    
                    critic_value = self.critic.model([ob_user, ob_assist, self.env_cell_mapping, asst_output_one_hot])
                    critic_value_history.append(critic_value)
                    
                    new_loc, reward_user, reward_assist, done = self.env.step(user_action, asst_action + 1, target_loc, curr_loc)
                    
                    next_ob_user = new_loc[:]
                    next_ob_user = next_ob_user + target_loc

                    ob_user = next_ob_user
                    prev_steps_assist = np.squeeze(ob_assist).tolist()[1:]
                    
                    rewards_history.append(reward_user)
                    episode_reward+=reward_user
                    
                    if done:
                        reached += 1

                running_reward = 0.05 * episode_reward + (1 - 0.05) * running_reward

                discounted_sum = 0
                for r in rewards_history[::-1]:
                    discounted_sum = r + self.gamma * discounted_sum
                    returns.append(discounted_sum)
                returns.reverse()
                
#                 returns = np.array(returns)
#                 returns = (returns - np.mean(returns)) / (np.std(returns) + self.eps)
#                 returns = returns.tolist()
                
                critic_losses = []
                user_losses = []
                asst_losses = []
                
                for log_prob_user, log_prob_asst, val, ret in zip(user_action_probs_history, asst_action_probs_history, critic_value_history,\
                                                                 returns):
                    diff = ret - val
                    user_losses.append(-log_prob_user*diff)
                    asst_losses.append(-log_prob_asst*diff)
                    critic_losses.append(self.huber_loss(tf.expand_dims(val, 0), tf.expand_dims(ret, 0)))

                user_loss = sum(user_losses)
                asst_loss = sum(asst_losses)
                critic_loss = sum(critic_losses)

            grads = tape.gradient(user_loss, self.user_actor.model.trainable_variables)
            self.optimizer_actors.apply_gradients(zip(grads, self.user_actor.model.trainable_variables))

            grads = tape.gradient(asst_loss, self.asst_actor.model.trainable_variables)
            self.optimizer_actors.apply_gradients(zip(grads, self.asst_actor.model.trainable_variables))

            grads = tape.gradient(critic_loss, self.critic.model.trainable_variables)
            self.optimizer_critic.apply_gradients(zip(grads, self.critic.model.trainable_variables))
            
            if epoch%100 == 0:
                print(running_reward)
            
            if epoch and epoch%100 == 0:
                print(reached)
                if reached>=best:
                    best = reached
                    tf.keras.models.save_model(self.user_actor.model, 'user.h5')
                    tf.keras.models.save_model(self.asst_actor.model, 'asst.h5')
                    tf.keras.models.save_model(self.critic.model, 'critic.h5')
                reached = 0
                
                if epoch%200:
                    self.trial()
                
                
    def trial(self):
        env = self.env
        max_steps = 40
        done = False
        episode_reward = 0
        start, dest = env.give_start_dest()
        ob_user = [start[0], start[1], dest[0], dest[1]]
        prev_steps_assist = []
        prev_steps_assist = give_prev_steps(prev_steps_assist, self.memory_len)
        step = 0
        episode_reward = 0

        while not done and step<max_steps:
            curr_loc = ob_user[:2]
            target_loc = ob_user[2:4]
            step+=1
            print(ob_user)

            ob_user = np.array(ob_user)[np.newaxis]
            user_probs = self.user_actor.model(ob_user)
            user_action = np.argmax(np.squeeze(user_probs))

            action_user_one_hot = make_one_hot(user_action, 4)

            ob_assist = [action_user_one_hot + curr_loc] 
            ob_assist = prev_steps_assist + ob_assist
            ob_assist = np.array(ob_assist)[np.newaxis]

            asst_probs = self.asst_actor.model([ob_assist, self.env_cell_mapping])
            print(asst_probs)
            asst_action = np.argmax(np.squeeze(asst_probs)) 
            new_loc, reward_user, reward_assist, done = self.env.step(user_action, asst_action + 1, target_loc, curr_loc)

            next_ob_user = new_loc[:]
            next_ob_user = next_ob_user + target_loc

            ob_user = next_ob_user
            prev_steps_assist = np.squeeze(ob_assist).tolist()[1:]
            episode_reward+=reward_user




In [4]:
agent = Agent()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 4)]          0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice (Tens [(None, 2)]          0           input_1[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_strided_slice_1 (Te [(None, 2)]          0           input_1[0][0]                    
__________________________________________________________________________________________________
subtract (Subtract)             (None, 2)            0           tf_op_layer_strided_slice[0][0]  
                                                                 tf_op_layer_strided_sl

In [5]:
agent.learn()

  0%|                                                                            | 1/100000 [00:03<87:22:49,  3.15s/it]

-0.75


  0%|                                                                          | 100/100000 [04:12<79:44:32,  2.87s/it]

-46.65408750724117
20
[0.6, 0.2, 0.5, 0.7]
tf.Tensor([[0.26629087 0.24951543 0.23303398 0.25115973]], shape=(1, 4), dtype=float32)
[0.6, 0.1, 0.5, 0.7]
tf.Tensor([[0.25087932 0.26009727 0.23454188 0.2544815 ]], shape=(1, 4), dtype=float32)
[0.6, 0.0, 0.5, 0.7]
tf.Tensor([[0.23959439 0.26975098 0.23352346 0.2571312 ]], shape=(1, 4), dtype=float32)
[0.6, 0.0, 0.5, 0.7]
tf.Tensor([[0.2331702  0.27269793 0.23253459 0.26159728]], shape=(1, 4), dtype=float32)
[0.6, 0.0, 0.5, 0.7]
tf.Tensor([[0.23341382 0.2726923  0.23237285 0.26152107]], shape=(1, 4), dtype=float32)
[0.6, 0.0, 0.5, 0.7]
tf.Tensor([[0.233458   0.27270854 0.23234154 0.26149192]], shape=(1, 4), dtype=float32)
[0.6, 0.0, 0.5, 0.7]
tf.Tensor([[0.233458   0.27270854 0.23234154 0.26149192]], shape=(1, 4), dtype=float32)
[0.6, 0.0, 0.5, 0.7]
tf.Tensor([[0.233458   0.27270854 0.23234154 0.26149192]], shape=(1, 4), dtype=float32)
[0.6, 0.0, 0.5, 0.7]
tf.Tensor([[0.233458   0.27270854 0.23234154 0.26149192]], shape=(1, 4), dtype=float3

  0%|                                                                          | 101/100000 [04:16<84:12:45,  3.03s/it]

...

-9.370370606053168
94
[0.2, 0.9, 0.5, 0.7]
tf.Tensor([[2.9173485e-04 9.2330861e-01 7.4175812e-02 2.2238006e-03]], shape=(1, 4), dtype=float32)
[0.4, 0.9, 0.5, 0.7]
tf.Tensor([[2.9272932e-04 9.2752051e-01 7.0435882e-02 1.7509214e-03]], shape=(1, 4), dtype=float32)
[0.4, 0.7, 0.5, 0.7]
tf.Tensor([[3.2635653e-04 9.1305161e-01 8.5054003e-02 1.5680395e-03]], shape=(1, 4), dtype=float32)
[0.6, 0.7, 0.5, 0.7]
tf.Tensor([[6.3747622e-04 8.9291102e-01 1.0524621e-01 1.2052964e-03]], shape=(1, 4), dtype=float32)
[0.4, 0.7, 0.5, 0.7]
tf.Tensor([[5.7313143e-04 8.8107747e-01 1.1705275e-01 1.2966830e-03]], shape=(1, 4), dtype=float32)
[0.6, 0.7, 0.5, 0.7]
tf.Tensor([[0.00093695 0.88956505 0.10834676 0.00115127]], shape=(1, 4), dtype=float32)
[0.4, 0.7, 0.5, 0.7]
tf.Tensor([[0.00097896 0.8662162  0.13151091 0.00129403]], shape=(1, 4), dtype=float32)
[0.6, 0.7, 0.5, 0.7]
tf.Tensor([[0.00093695 0.88956505 0.10834676 0.00115127]], shape=(1, 4), dtype=float32)
[0.4, 0.7, 0.5, 0.7]
tf.Tensor([[0.00097896 0.

 20%|██████████████                                                        | 20101/100000 [5:45:19<35:23:50,  1.59s/it]


[0.6, 0.7, 0.5, 0.7]
tf.Tensor([[0.00093695 0.88956505 0.10834676 0.00115127]], shape=(1, 4), dtype=float32)
[0.4, 0.7, 0.5, 0.7]
tf.Tensor([[0.00097896 0.8662162  0.13151091 0.00129403]], shape=(1, 4), dtype=float32)
[0.6, 0.7, 0.5, 0.7]
tf.Tensor([[0.00093695 0.88956505 0.10834676 0.00115127]], shape=(1, 4), dtype=float32)
[0.4, 0.7, 0.5, 0.7]
tf.Tensor([[0.00097896 0.8662162  0.13151091 0.00129403]], shape=(1, 4), dtype=float32)
[0.6, 0.7, 0.5, 0.7]
tf.Tensor([[0.00093695 0.88956505 0.10834676 0.00115127]], shape=(1, 4), dtype=float32)
[0.4, 0.7, 0.5, 0.7]
tf.Tensor([[0.00097896 0.8662162  0.13151091 0.00129403]], shape=(1, 4), dtype=float32)
[0.6, 0.7, 0.5, 0.7]
tf.Tensor([[0.00093695 0.88956505 0.10834676 0.00115127]], shape=(1, 4), dtype=float32)
[0.4, 0.7, 0.5, 0.7]
tf.Tensor([[0.00097896 0.8662162  0.13151091 0.00129403]], shape=(1, 4), dtype=float32)
[0.6, 0.7, 0.5, 0.7]
tf.Tensor([[0.00093695 0.88956505 0.10834676 0.00115127]], shape=(1, 4), dtype=float32)
[0.4, 0.7, 0.5, 0.

 20%|██████████████▏                                                       | 20190/100000 [5:46:43<22:50:34,  1.03s/it]


KeyboardInterrupt: 

In [ ]:
agent.trial()